In [132]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)

from numba import jit, prange
from tqdm.auto import tqdm

from statsmodels.tsa.stattools import acf, pacf

from scipy.optimize import curve_fit

import imageio

We start with a random configuration:

In [133]:
@jit(nopython=True)
def init_state(N):
  '''
  Return a random spin configuration in a 2d square lattice
  '''
  return np.random.choice(np.array([-1,1]),size=(N,N))

## Metropolis-Hastings algorithm

In [134]:
@jit(nopython=True)
def energy(state):
  '''
  Energy of a given configuration
  '''
  N = state.shape[0]
  energy = 0
  
  # Loop over all sites
  for i in prange(N):
    for j in prange(N):

      # get nearest neighbours states
      nn = state[(i-1)%N, j] + state[(i+1)%N, j] + state[i, (j-1)%N] + state[i, (j+1)%N]
      # compute site energy
      energy -=  state[i,j] * nn

  # Divide energy by 2, since we count each site contribution to each of the 2 neighbours 
  return energy / 2 

In [135]:
@jit(nopython=True)
def glauber(state, beta):
  '''
  Spin-flip according to Metropolis-Hastings algorithm
  '''
  beta = 1
  N = state.shape[0]

  for n in prange(N**2):

      # choose a spin site with uniform probability
      i, j = np.random.randint(N), np.random.randint(N)

      # get sum of nearest neighbour values in the lattice
      nn = state[(i-1)%N, j] + state[(i+1)%N, j] + state[i, (j-1)%N] + state[i, (j+1)%N] # the modulo % operation takes care of the boundaries condition
      
      # get delta energy
      delta = 2 * state[i,j] * nn

      # Metropolis
      rand = np.random.random()
      prob = np.exp(- beta * delta)
      print(f"rand: {rand} \t  prob_spin: {prob}")
      if delta < 0 or rand < np.exp(- beta * delta):
        state[i,j] = -1. * state[i,j]
        
  return state

### Example of single spin flip

In [140]:
@jit(nopython=True)
def glauber(state, beta):
  '''
  Spin-flip according to Metropolis-Hastings algorithm
  '''
  beta = 1
  N = state.shape[0]

  for n in prange(N**2):

      # choose a spin site with uniform probability
      i, j = np.random.randint(N), np.random.randint(N)

      # get sum of nearest neighbour values in the lattice
      nn = state[(i-1)%N, j] + state[(i+1)%N, j] + state[i, (j-1)%N] + state[i, (j+1)%N] # the modulo % operation takes care of the boundaries condition
      
      # get delta energy
      delta = 2 * state[i,j] * nn

      # Metropolis
      rand = np.random.rand()
      prob = np.exp(- beta * delta)
      print(prob)
      if delta < 0 or rand < 1/np.exp(- beta * delta):
        state[i,j] = -1. * state[i,j]
        
  return state


def Ising(N, beta, eq_steps = int(1e4), steps = int(1e6), order = False, annealing = False, cluster = False, verbose=True):
    '''
    Run a simulation of a 2d Ising model

    Parameters:
    - N: number of sites per dimension
    - beta
    - eq_steps: initial steps for the equilibration phase
    - steps: effective steps of the simulation
    - order: generate a ordered configuration with uniform magnetisation
    - annealing: using simulating annealing during equilibration
    - cluster: to use the Wolff algorithm
    - verbose: more detailed output during simulation
    
    Output:
    - E_mean: Mean energy 
    - M_mean: Mean magnetization
    - Cs: Specific heat
    - Chis: Susceptibility
    - E_var: Energy error
    - M_var: Magnetisation error
    - E_eq: Energy on equilibration
    - M_eq: Magnetisation on equilibration
    - E: Energy on simulation
    - M Magnetisation on simulation

    '''

  # Init state
    lst = []
    curr_state = np.random.choice(np.array([-1,1]),size=(N,N))
    print(curr_state)
        
    lst.append(curr_state)
  
    if verbose:
        for i in tqdm(range(steps)):
            E[i] = energy(curr_state)
            M[i] = np.sum(curr_state)
            curr_state = glauber(curr_state, beta)
            lst.append(curr_state)

  ### Return results
    return lst
    
N = 20 #size of lattice
eq_steps = 1000 #define number of equilibration steps
steps = 100 #define number of simulation steps
lst = Ising(N, 1, eq_steps, steps, order = False)


import pandas as pd


mm = [tuple(i.flatten()) for i in lst]
test = pd.DataFrame({"a": mm})
counts = test.groupby("a")["a"].count()
counts = [i for i in counts]
print(counts)
print(test.groupby("a")["a"].count())

[[ 1  1 -1  1  1 -1  1  1  1 -1  1  1 -1 -1 -1  1 -1 -1 -1  1]
 [ 1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1]
 [-1  1 -1 -1 -1  1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1  1 -1  1]
 [ 1  1  1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1  1 -1  1  1  1  1]
 [-1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1]
 [ 1  1 -1  1 -1 -1  1  1 -1  1 -1  1 -1 -1  1 -1  1 -1 -1  1]
 [ 1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1  1  1 -1  1 -1 -1]
 [-1 -1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1  1]
 [-1 -1  1 -1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1  1  1  1  1 -1]
 [ 1  1  1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1 -1]
 [ 1 -1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1 -1 -1]
 [ 1  1 -1  1 -1  1 -1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1  1 -1 -1  1 -1 -1  1 -1 -1  1  1  1 -1 -1  1 -1]
 [-1 -1 -1  1  1  1  1 -1  1  1  1 -1 -1  1 -1 -1  1 -1  1 -1]
 [ 1 -1  1  1  1 -1  1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1 -1 -1]
 [ 1 -1 -1 -1  1 -1  1  1  1 -1  1  1 -1 -1  1  1 -1  1

  0%|          | 0/100 [00:00<?, ?it/s]

54.598150033144236
0.01831563888873418
54.598150033144236
0.01831563888873418
54.598150033144236
0.01831563888873418
54.598150033144236
0.01831563888873418
1.0
1.0
0.01831563888873418
1.0
0.01831563888873418
0.01831563888873418
1.0
0.01831563888873418
0.01831563888873418
0.01831563888873418
0.01831563888873418
1.0
0.01831563888873418
0.01831563888873418
54.598150033144236
54.598150033144236
1.0
0.01831563888873418
0.01831563888873418
1.0
0.01831563888873418
0.01831563888873418
54.598150033144236
1.0
0.00033546262790251185
0.01831563888873418
54.598150033144236
2980.9579870417283
54.598150033144236
1.0
1.0
54.598150033144236
0.00033546262790251185
0.01831563888873418
1.0
54.598150033144236
0.01831563888873418
54.598150033144236
0.01831563888873418
54.598150033144236
2980.9579870417283
1.0
1.0
54.598150033144236
0.01831563888873418
1.0
54.598150033144236
0.01831563888873418
54.598150033144236
54.598150033144236
0.01831563888873418
1.0
54.598150033144236
1.0
0.01831563888873418
0.01831563

# **SIMULATION**

Before starting with the analysis of the system we can check if the model behaves as expected, testing the outputs for different inputs.
<br>
Here we plot the energy and the magnetisation during the equilibration for some values of temperature.

In [137]:
import pandas as pd


mm = [tuple(i.flatten()) for i in lst]
test = pd.DataFrame({"a": mm})
counts = test.groupby("a")["a"].count()
counts = [i for i in counts]
print(counts)
print(test.groupby("a")["a"].count())

[101]
a
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...)    101
Name: a, dtype: int64


In [138]:
print(test.groupby("a")["a"].count())

a
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...)    101
Name: a, dtype: int64
